In [1]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

from glob import glob

from tqdm import tqdm
import pykep as pk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.neighbors import NearestNeighbors

from vtk import vtkXMLUnstructuredGridReader
from vtk.util import numpy_support as VN

from mpl_toolkits import mplot3d
%matplotlib notebook

dt = 10 #timestep of the inspected simulations, affects time in plots
iterations = 3153600 # for time computation as well

# PK epochs of simulation time
starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000')
end_t = pk.epoch(starting_t.mjd2000 + iterations * dt * pk.SEC2DAY)
total_days = end_t.mjd - starting_t.mjd

In [ ]:
# Load the original dataset
init_r = np.loadtxt("../data/pos.csv",delimiter=",")
init_v = np.loadtxt("../data/v.csv",delimiter=",")
init_planets = []
for r,v in zip(init_r,init_v):
    init_planets.append(pk.planet.keplerian(starting_t,r * 1000.0,v * 1000.0,pk.MU_EARTH,1.,1.,1.))

In [11]:
# Load some simulation results
# files = glob("../results/1y_kep_thresholds_test/*.vtu")
files = glob("../results/1y_all_thresholds_test/*.vtu")
files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

In [ ]:
rs,vs = [],[] #will hold r,v for whole simulation

# load vtks
for file in tqdm(files):
    reader = vtkXMLUnstructuredGridReader()
    reader.SetFileName(file)
    reader.Update()

    data = reader.GetOutput()

    v = VN.vtk_to_numpy(data.GetPointData().GetArray('velocity')).astype("double")
    r = VN.vtk_to_numpy(data.GetPoints().GetData()).astype("double")
    rs.append(r)
    vs.append(v)

In [ ]:
# Threshold for conjunction tracking as used in simulation
thresholds = [1,5,10,20,25,50,75,100,250,500]

In [ ]:
# Load the file tracking the conjunctions
# conj = pd.read_csv("../results/1y_kep_thresholds_test/conjunctionCounts.csv",names=thresholds)
conj = pd.read_csv("../results/1y_all_thresholds_test/conjunctionCounts.csv")

In [ ]:
# Sort ascending by conjunction distance, then drop all but the first to get closest encounter
conj = conj.sort_values('SquaredDistance', ascending=True)
unique_conj = conj.drop_duplicates(subset=["P1","P2"],keep="first")
unique_conj["Distance"] = np.sqrt(unique_conj.SquaredDistance) * 1000. # Compute distance in meters

In [ ]:
unique_conj.sort_values('Iteration', ascending=True)[unique_conj.Iteration < 50000]

In [ ]:
# Compute conjunction counts at specific points in the simulation
stepsize = 100
# Time axis
t = np.linspace(0,end_t.mjd - starting_t.mjd,stepsize)
it = np.linspace(0,iterations,stepsize)
summed_conjs = [[] for _ in thresholds]
for i in tqdm(it):
    for idx,threshold in enumerate(thresholds):
        count = len(unique_conj[(unique_conj.Iteration < i) & (unique_conj.Distance < threshold)])
        summed_conjs[idx].append(count)

In [ ]:
fig = plt.figure(figsize=(9,5),dpi=100)
fig.patch.set_facecolor('white')

# Iterate over thresholds and plot for each
for idx,row in enumerate(thresholds):
    plt.plot(t,summed_conjs[idx],linewidth=3)
    
plt.legend([str(t) + "m" for t in thresholds],loc='upper center', bbox_to_anchor=(1.1,0.8), ncol=1, fancybox=True, shadow=True)
plt.title("Conjunction Thresholds Comparison")
plt.xlabel("Days")
plt.ylabel("# of Conjunctions")
plt.gca().set_yscale("log")
plt.tight_layout()

In [ ]:
# Compute min, max and mean of orbital elements over simulation
min_elements = [[],[],[],[],[],[]]
mean_elements = [[],[],[],[],[],[]]
max_elements = [[],[],[],[],[],[]]

for v_it,r_it in tqdm(zip(vs,rs),total=len(vs)):
    elements = [[],[],[],[],[],[]]
    for v,r in zip(v_it,r_it):
        a,e,i,W,w,E = pk.ic2par(r,v, pk.MU_EARTH)
        elements[0].append(abs(a))
        elements[1].append(abs(e))
        elements[2].append(abs(i))
        elements[3].append(abs(W))
        elements[4].append(abs(w))
        elements[5].append(abs(E))
    for i in range(6):
        min_elements[i].append(np.min(elements[i]))
        mean_elements[i].append(np.mean(elements[i]))
        max_elements[i].append(np.max(elements[i]))

In [ ]:
#t ime axis of the simulation
t = np.linspace(0,end_t.mjd - starting_t.mjd,len(mean_elements[0]))

# Plot for each orbital element
for idx,element in enumerate(["a","e","i","W","w","E"]):
    fig = plt.figure(figsize=(8,4),dpi=150)
    fig.patch.set_facecolor('white')
    plt.plot(t,mean_elements[idx],linewidth=1)
#     plt.plot(t,min_elements[idx],linewidth=1)
#     plt.plot(t,max_elements[idx],linewidth=1)
    plt.legend(["mean","min","max"],loc='upper center', bbox_to_anchor=(1.1,0.8), ncol=1, fancybox=True, shadow=True)
    plt.title("Evolution of "+element)
    plt.xlabel("Days")
    plt.ylabel(element)
#         plt.gca().set_yscale("log")

In [ ]:
# Compute a KNN to get distance to nearest neighbors over simulation

all_distances = []

r_subset = rs[::10]

for r_it in tqdm(r_subset,total=len(r_subset)):
    elements = [[],[],[],[],[],[]]
    knn = NearestNeighbors(n_neighbors=2).fit(r_it)
    distances,_ = knn.kneighbors(r_it)
    distances = distances[:,1] / 1000 # convert to km
    all_distances.append(distances)
    
all_distances = np.asarray(all_distances)

In [ ]:
# Subsample to look only at those below some threshold
small_distances = []
max_dist = 1000
for dist in all_distances:
    small_distances.append(dist[dist < max_dist])

In [ ]:
# Compute distributions for each iteration
bins = 64
x = np.linspace(0, max_dist, bins)
y = np.linspace(0, total_days, len(all_distances))

X, Y = np.meshgrid(x, y)
Z = []
for dist in tqdm(small_distances):
    hist,bin_vals = np.histogram(dist, bins = x,density=False)
    hist = np.cumsum(hist)
    hist = np.concatenate([hist,[hist[-1]]]) # last value remains for bucket
    Z.append(hist)
x = np.concatenate([[0],x])
Z = np.asarray(Z)

In [ ]:
# Create beautiful plots
fig = plt.figure(figsize = (6,6),dpi=150)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z,  rstride=1, cstride=1, cmap="plasma", edgecolor='none')
ax.view_init(elev=20., azim=120)
ax.set_xlabel('Closest Distance [km]')
ax.set_ylabel('Days')
ax.set_zlabel('Cumulative Frequency');
ax.set_xlim([0,max_dist])

In [ ]:
# trace generated using paraview version 5.10.0-RC1

#### import the simple module from the paraview
from paraview.simple import *
#### disable automatic camera reset on 'Show'
paraview.simple._DisableFirstRenderCameraReset()

# create a new 'XML Unstructured Grid Reader'
output_ = XMLUnstructuredGridReader(registrationName='output_*', FileName=files)
output_.PointArrayStatus = ['characteristic-length', 'mass', 'area', 'area-to-mass', 'velocity', 'ejection-velocity']

# get animation scene
animationScene1 = GetAnimationScene()

# update animation scene based on data timesteps
animationScene1.UpdateAnimationUsingDataTimeSteps()

# Properties modified on output_
output_.TimeArray = 'None'

# get active view
renderView1 = GetActiveViewOrCreate('RenderView')

# show data in view
output_Display = Show(output_, renderView1, 'UnstructuredGridRepresentation')

# trace defaults for the display properties.
output_Display.Representation = 'Surface'
output_Display.ColorArrayName = [None, '']
output_Display.SelectTCoordArray = 'None'
output_Display.SelectNormalArray = 'None'
output_Display.SelectTangentArray = 'None'
output_Display.OSPRayScaleArray = 'area'
output_Display.OSPRayScaleFunction = 'PiecewiseFunction'
output_Display.SelectOrientationVectors = 'None'
output_Display.ScaleFactor = 1632146.4500000002
output_Display.SelectScaleArray = 'None'
output_Display.GlyphType = 'Arrow'
output_Display.GlyphTableIndexArray = 'None'
output_Display.GaussianRadius = 81607.3225
output_Display.SetScaleArray = ['POINTS', 'area']
output_Display.ScaleTransferFunction = 'PiecewiseFunction'
output_Display.OpacityArray = ['POINTS', 'area']
output_Display.OpacityTransferFunction = 'PiecewiseFunction'
output_Display.DataAxesGrid = 'GridAxesRepresentation'
output_Display.PolarAxes = 'PolarAxesRepresentation'
output_Display.ScalarOpacityUnitDistance = 28128178.729679402
output_Display.OpacityArrayName = ['POINTS', 'area']

# init the 'PiecewiseFunction' selected for 'ScaleTransferFunction'
output_Display.ScaleTransferFunction.Points = [0.025235028937458992, 0.0, 0.5, 0.0, 0.025238843634724617, 1.0, 0.5, 0.0]

# init the 'PiecewiseFunction' selected for 'OpacityTransferFunction'
output_Display.OpacityTransferFunction.Points = [0.025235028937458992, 0.0, 0.5, 0.0, 0.025238843634724617, 1.0, 0.5, 0.0]

# reset view to fit data
renderView1.ResetCamera(False)

# get the material library
materialLibrary1 = GetMaterialLibrary()

# update the view to ensure updated data information
renderView1.Update()

# create a new 'Temporal Particles To Pathlines'
temporalParticlesToPathlines1 = TemporalParticlesToPathlines(registrationName='TemporalParticlesToPathlines1', Input=output_,
    Selection=None)

# show data in view
temporalParticlesToPathlines1Display = Show(temporalParticlesToPathlines1, renderView1, 'GeometryRepresentation')

# get color transfer function/color map for 'TrailId'
trailIdLUT = GetColorTransferFunction('TrailId')

# trace defaults for the display properties.
temporalParticlesToPathlines1Display.Representation = 'Surface'
temporalParticlesToPathlines1Display.ColorArrayName = ['POINTS', 'TrailId']
temporalParticlesToPathlines1Display.LookupTable = trailIdLUT
temporalParticlesToPathlines1Display.SelectTCoordArray = 'None'
temporalParticlesToPathlines1Display.SelectNormalArray = 'None'
temporalParticlesToPathlines1Display.SelectTangentArray = 'None'
temporalParticlesToPathlines1Display.OSPRayScaleArray = 'TrailId'
temporalParticlesToPathlines1Display.OSPRayScaleFunction = 'PiecewiseFunction'
temporalParticlesToPathlines1Display.SelectOrientationVectors = 'None'
temporalParticlesToPathlines1Display.ScaleFactor = 1542217.55
temporalParticlesToPathlines1Display.SelectScaleArray = 'TrailId'
temporalParticlesToPathlines1Display.GlyphType = 'Arrow'
temporalParticlesToPathlines1Display.GlyphTableIndexArray = 'TrailId'
temporalParticlesToPathlines1Display.GaussianRadius = 77110.8775
temporalParticlesToPathlines1Display.SetScaleArray = ['POINTS', 'TrailId']
temporalParticlesToPathlines1Display.ScaleTransferFunction = 'PiecewiseFunction'
temporalParticlesToPathlines1Display.OpacityArray = ['POINTS', 'TrailId']
temporalParticlesToPathlines1Display.OpacityTransferFunction = 'PiecewiseFunction'
temporalParticlesToPathlines1Display.DataAxesGrid = 'GridAxesRepresentation'
temporalParticlesToPathlines1Display.PolarAxes = 'PolarAxesRepresentation'

# init the 'PiecewiseFunction' selected for 'ScaleTransferFunction'
temporalParticlesToPathlines1Display.ScaleTransferFunction.Points = [0.0, 0.0, 0.5, 0.0, 17300.0, 1.0, 0.5, 0.0]

# init the 'PiecewiseFunction' selected for 'OpacityTransferFunction'
temporalParticlesToPathlines1Display.OpacityTransferFunction.Points = [0.0, 0.0, 0.5, 0.0, 17300.0, 1.0, 0.5, 0.0]

# hide data in view
Hide(output_, renderView1)

# show color bar/color legend
temporalParticlesToPathlines1Display.SetScalarBarVisibility(renderView1, True)

# show data in view
temporalParticlesToPathlines1Display_1 = Show(OutputPort(temporalParticlesToPathlines1, 1), renderView1, 'GeometryRepresentation')

# trace defaults for the display properties.
temporalParticlesToPathlines1Display_1.Representation = 'Surface'
temporalParticlesToPathlines1Display_1.ColorArrayName = [None, '']
temporalParticlesToPathlines1Display_1.SelectTCoordArray = 'None'
temporalParticlesToPathlines1Display_1.SelectNormalArray = 'None'
temporalParticlesToPathlines1Display_1.SelectTangentArray = 'None'
temporalParticlesToPathlines1Display_1.OSPRayScaleArray = 'area'
temporalParticlesToPathlines1Display_1.OSPRayScaleFunction = 'PiecewiseFunction'
temporalParticlesToPathlines1Display_1.SelectOrientationVectors = 'None'
temporalParticlesToPathlines1Display_1.ScaleFactor = 1542217.55
temporalParticlesToPathlines1Display_1.SelectScaleArray = 'None'
temporalParticlesToPathlines1Display_1.GlyphType = 'Arrow'
temporalParticlesToPathlines1Display_1.GlyphTableIndexArray = 'None'
temporalParticlesToPathlines1Display_1.GaussianRadius = 77110.8775
temporalParticlesToPathlines1Display_1.SetScaleArray = ['POINTS', 'area']
temporalParticlesToPathlines1Display_1.ScaleTransferFunction = 'PiecewiseFunction'
temporalParticlesToPathlines1Display_1.OpacityArray = ['POINTS', 'area']
temporalParticlesToPathlines1Display_1.OpacityTransferFunction = 'PiecewiseFunction'
temporalParticlesToPathlines1Display_1.DataAxesGrid = 'GridAxesRepresentation'
temporalParticlesToPathlines1Display_1.PolarAxes = 'PolarAxesRepresentation'

# init the 'PiecewiseFunction' selected for 'ScaleTransferFunction'
temporalParticlesToPathlines1Display_1.ScaleTransferFunction.Points = [0.025235028937458992, 0.0, 0.5, 0.0, 0.025238843634724617, 1.0, 0.5, 0.0]

# init the 'PiecewiseFunction' selected for 'OpacityTransferFunction'
temporalParticlesToPathlines1Display_1.OpacityTransferFunction.Points = [0.025235028937458992, 0.0, 0.5, 0.0, 0.025238843634724617, 1.0, 0.5, 0.0]

# hide data in view
Hide(output_, renderView1)

# update the view to ensure updated data information
renderView1.Update()

# get opacity transfer function/opacity map for 'TrailId'
trailIdPWF = GetOpacityTransferFunction('TrailId')

# Properties modified on renderView1
renderView1.UseColorPaletteForBackground = 0

# Properties modified on renderView1
renderView1.Background = [0.0, 0.0, 0.0]

# Properties modified on temporalParticlesToPathlines1
temporalParticlesToPathlines1.MaxStepDistance = [1000000000.0, 1000000000.0, 1000000000.0]

# update the view to ensure updated data information
renderView1.Update()

#================================================================
# addendum: following script captures some of the application
# state to faithfully reproduce the visualization during playback
#================================================================

# get layout
layout1 = GetLayout()

#--------------------------------
# saving layout sizes for layouts

# layout/tab size in pixels
layout1.SetSize(1391, 836)

#-----------------------------------
# saving camera placements for views

# current camera placement for renderView1
renderView1.CameraPosition = [2.735207743104825, 0.3529300321278762, 53664671.90629359]
renderView1.CameraFocalPoint = [2.735207743104826, 0.3529300321278762, 53664663.80881577]
renderView1.CameraParallelScale = 1.7320508075688772

#--------------------------------------------
# uncomment the following to render all views
# RenderAllViews()
# alternatively, if you want to write images, you can use SaveScreenshot(...).